In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import os

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

def create_keras_model():
    
    model = Sequential()
    model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

    return model

In [3]:
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32') / 255
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [4]:
model = create_keras_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        9280      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                        

In [5]:

history = model.fit(x=X_train,y=y_train, epochs=2, validation_split=0.01,verbose=False)
_,testing_accuracy = model.evaluate(X_test, y_test,verbose=False)
print("total training samples: {}\t training accuracy: {}\t validation accuracy: {}\t testing accuracy: {}".format(int(X_train.shape[0]*0.9),
                                                                                                               history.history['accuracy'][-1],
                                                                                                               history.history['val_accuracy'][-1],
                                                                                                              testing_accuracy))

total training samples: 54000	 training accuracy: 0.6037542223930359	 validation accuracy: 0.8366666436195374	 testing accuracy: 0.7910000085830688


In [6]:
def uncertain_idx(model,x,new_samples):
    pred = model.predict(x)
    idxs = np.argsort(np.squeeze(pred).var(axis=1))[:new_samples]
    return idxs,pred

def get_ambigious_labels(model,X_reserve,y_reserve,X_pool,y_pool,new_samples=200):

    idx,pred = uncertain_idx(model, X_reserve,new_samples)

    X_new_points = X_reserve[idx]
    y_new_points = y_reserve[idx]

    X_pool = np.concatenate((X_pool,X_new_points),axis=0)
    y_pool = np.concatenate((y_pool,y_new_points),axis=0)
    
    X_reserve = np.delete(X_reserve,idx,axis=0)
    y_reserve = np.delete(y_reserve,idx,axis=0)
    
    return X_reserve,y_reserve,X_pool,y_pool

In [7]:
def model_fit(X_pool,y_pool):
    model = create_keras_model()
    hist = model.fit(X_pool,y_pool,epochs=2,validation_split=0.01,verbose=False)
    return model,hist 

In [8]:
X_pool, X_reserve, y_pool, y_reserve = train_test_split(X_train, y_train, test_size = 0.4)
test_max = 0

for i in range(100):
    
    model, hist = model_fit(X_pool, y_pool)
    _,testing_accuracy = model.evaluate(X_test, y_test,verbose=False)
    
    if testing_accuracy > test_max:
        print("iteration: {}\t total training samples: {}\t training accuracy: {}\t validation accuracy: {}\t testing accuracy: {}".format(i, y_pool.shape[0],
                                                                                                                                       round(hist.history["accuracy"][-1],3),
                                                                                                                                       round(hist.history['val_accuracy'][-1],3),
                                                                                                                                        round(testing_accuracy,3)))
        test_max = testing_accuracy
        
    X_reserve,y_reserve,X_pool,y_pool = get_ambigious_labels(model, X_reserve, y_reserve, X_pool, y_pool,new_samples=50)

iteration: 0	 total training samples: 36000	 training accuracy: 0.485	 validation accuracy: 0.703	 testing accuracy: 0.712
744/744 [==============================] - 1s 986us/step
iteration: 5	 total training samples: 36250	 training accuracy: 0.476	 validation accuracy: 0.46	 testing accuracy: 0.72
724/724 [==============================] - 1s 947us/step
iteration: 18	 total training samples: 36900	 training accuracy: 0.498	 validation accuracy: 0.439	 testing accuracy: 0.727
682/682 [==============================] - 1s 920us/step
iteration: 45	 total training samples: 38250	 training accuracy: 0.486	 validation accuracy: 0.551	 testing accuracy: 0.728
677/677 [==============================] - 1s 961us/step
iteration: 48	 total training samples: 38400	 training accuracy: 0.474	 validation accuracy: 0.513	 testing accuracy: 0.739
633/633 [==============================] - 1s 977us/step
iteration: 76	 total training samples: 39800	 training accuracy: 0.564	 validation accuracy: 0.611	